# Computation of spatial routes, travel-times and travel-distances

In [2]:
#general libraries

from shapely.geometry import shape
from shapely.geometry import LineString
import polyline
from osgeo import ogr, osr
import geopandas as gpd
import pandas as pd
import json
import time


#specific to osrm approach
import requests

#specific to osmnx & networkx approach
import networkx as nx
import osmnx as ox


#import folium

# 1. OSRM to find fastest paths

OSRM is particularly useful when you want to calculate fastest routes (traffic independent conditions) and are looking for a free and open-source routing service. For more services by OSRM head to its [documentation](http://project-osrm.org/docs/v5.23.0/api/#). You can also set up your own OSRM server for bulkier requests. 

The script below uses the get_route() method discussed by [Yan (2020)](https://www.thinkdatascience.com/post/2020-03-03-osrm/osrm/).  The method is modified to collect the 'duration' attribute as well from the response string, in the output dictionary. For more options such as alternative routes, mode of transportation (driving, bike, foot) i.e. the speed profile, do have a look at the OSRM documentation. In this script, we use [driving (car)](https://github.com/Project-OSRM/osrm-backend/blob/master/profiles/car.lua) speeds profile. You can also create your own speed profiles.


In [6]:
#Read the csv with origin-destination pairs arranged row-wise. 
#In the table used in this example, o_long and o_lat are coordinates of origin point, similarly for destination

od_table = "ODMatrix_Settlements_NearestFacility.csv"
df = pd.read_csv(od_table)
df

,s_id,s_nm,s_tenability,s_wno,s_hh,s_pop,s_zno,s_addr,s_lat,s_lon,u_id,u_zno,u_wno,u_nm,u_addr,u_lat,u_lon,qneat_dist
0,S1-3-34,Periya Kasi Koil Kuppam,Tenable,3,430,1493,1,"Periya Kasi Koil Kuppam, Ward 3, THIRUVOTRIYUR...",13.198710,80.317870,UPHC1,1,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,2671.624352
1,S1-2-11,ChinnaKuppam,Untenable,2,239,786,1,"ChinnaKuppam, Ward 2, THIRUVOTRIYUR, Chennai, ...",13.207463,80.322746,UPHC1,1,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,1552.803335
2,S1-2-24,PeriyaKuppam,Untenable,2,280,918,1,"PeriyaKuppam, Ward 2, THIRUVOTRIYUR, Chennai, ...",13.212160,80.324166,UPHC1,1,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,1096.379136
3,S1-2-21,KattuKuppam,Tenable,2,334,1225,1,"KattuKuppam, Ward 2, THIRUVOTRIYUR, Chennai, T...",13.218668,80.319222,UPHC1,1,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,357.142240
4,S1-1-4,Nettukuppam,Untenable,1,403,1457,1,"Nettukuppam, Ward 1, THIRUVOTRIYUR, Chennai, T...",13.229021,80.328970,UPHC1,1,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,2343.963963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,S15-197-22,"Solingahanallur,Gandhi Street",Tenable,197,44,129,15,"Solingahanallur,Gandhi Street, Ward 197, SOZHA...",12.911636,80.231167,UPHC137,15,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2954.246249
389,S15-197-26,"Karapakkam , Mahatma Gandhi Street",Tenable,197,23,70,15,"Karapakkam , Mahatma Gandhi Street, Ward 197, ...",12.913490,80.229135,UPHC137,15,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2601.931062
390,S15-197-27,"Karapakkam,N S.K Street",Tenable,197,24,97,15,"Karapakkam,N S.K Street, Ward 197, SOZHANGANAL...",12.913490,80.229135,UPHC137,15,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2601.931062
391,S15-197-21,Karapakkam,Tenable,197,49,178,15,"Karapakkam, Ward 197, SOZHANGANALLUR, Chennai,...",12.919052,80.230034,UPHC137,15,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,1959.684015


In [8]:
#Method to send requests to OSRM server and parse the json response to collect distance, duration, and route information 
#and return a dictionary
def get_route(origin_lon, origin_lat, dest_lon, dest_lat):
    
    loc = "{},{};{},{}".format(origin_lon, origin_lat, dest_lon, dest_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    duration = res['routes'][0]['duration']
    
    out = {'route':routes, #a list of tuples of coordinates along the route
           'start_point':start_point, #list of float coords
           'end_point':end_point,
           'distance':distance, #in metres
           'duration': duration #in seconds
          }

    return out

In [9]:
def reverseTuple(lstOfTuple): 
      
    return [tup[::-1] for tup in lstOfTuple]

#extract_route() parses the list of coordinates in the dictionary returned by get_route() 
#and returns the route geometry essential for outputting spatial features
def extract_route(routes_dict):
    route_coords_list= routes_dict['route']    
    listnew = reverseTuple(route_coords_list)    
    return(LineString(listnew))

def extract_duration(routes_dict):
    return(routes_dict['duration'])

def extract_distance(routes_dict):
    return(routes_dict['distance'])

#You can also wrap above 3 methods into a single one and return a single tuple with the three elements

In [12]:
#applying the above methods to the dataframe to calculate and add attributes to the dataframe

start_time = time.time()


df['routes'] = df.apply(lambda x: get_route(x['s_lon'], x['s_lat'], x['u_lon'], x['u_lat']), axis=1)
df['geometry'] = df.apply(lambda x: extract_route(x['routes']), axis=1)
df['osrm_dur'] = df.apply(lambda x: extract_duration(x['routes']),axis=1)
df['osrm_dist'] = df.apply(lambda x: extract_distance(x['routes']),axis=1)


print("Time taken: ", (time.time() - start_time), "seconds")
df

Time taken:  197.79084181785583 seconds


,s_id,s_nm,s_tenability,s_wno,s_hh,s_pop,s_zno,s_addr,s_lat,s_lon,...,u_wno,u_nm,u_addr,u_lat,u_lon,qneat_dist,routes,geometry,osrm_dur,osrm_dist
0,S1-3-34,Periya Kasi Koil Kuppam,Tenable,3,430,1493,1,"Periya Kasi Koil Kuppam, Ward 3, THIRUVOTRIYUR...",13.198710,80.317870,...,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,2671.624352,"{'route': [(13.19871, 80.31787), (13.1994, 80....","LINESTRING (80.31787 13.19871, 80.31585 13.199...",219.7,2670.5
1,S1-2-11,ChinnaKuppam,Untenable,2,239,786,1,"ChinnaKuppam, Ward 2, THIRUVOTRIYUR, Chennai, ...",13.207463,80.322746,...,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,1552.803335,"{'route': [(13.2073, 80.3227), (13.20758, 80.3...","LINESTRING (80.3227 13.2073, 80.32172 13.20758...",207.1,1534.4
2,S1-2-24,PeriyaKuppam,Untenable,2,280,918,1,"PeriyaKuppam, Ward 2, THIRUVOTRIYUR, Chennai, ...",13.212160,80.324166,...,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,1096.379136,"{'route': [(13.21207, 80.32411), (13.21219, 80...","LINESTRING (80.32411 13.21207, 80.323890000000...",170.1,1102.7
3,S1-2-21,KattuKuppam,Tenable,2,334,1225,1,"KattuKuppam, Ward 2, THIRUVOTRIYUR, Chennai, T...",13.218668,80.319222,...,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,357.142240,"{'route': [(13.21863, 80.31922), (13.21859, 80...","LINESTRING (80.31922 13.21863, 80.31903 13.218...",52.5,351.4
4,S1-1-4,Nettukuppam,Untenable,1,403,1457,1,"Nettukuppam, Ward 1, THIRUVOTRIYUR, Chennai, T...",13.229021,80.328970,...,2,Kathivakkam,"Urban Primary Health Centre, Kathivakkam, No 3...",13.216133,80.318177,2343.963963,"{'route': [(13.22892, 80.32914), (13.22872, 80...","LINESTRING (80.32914 13.22892, 80.329040000000...",228.4,2365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,S15-197-22,"Solingahanallur,Gandhi Street",Tenable,197,44,129,15,"Solingahanallur,Gandhi Street, Ward 197, SOZHA...",12.911636,80.231167,...,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2954.246249,"{'route': [(12.91163, 80.23117), (12.91182, 80...","LINESTRING (80.23117000000001 12.91163, 80.229...",400.4,4885.9
389,S15-197-26,"Karapakkam , Mahatma Gandhi Street",Tenable,197,23,70,15,"Karapakkam , Mahatma Gandhi Street, Ward 197, ...",12.913490,80.229135,...,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2601.931062,"{'route': [(12.91349, 80.22913), (12.92888, 80...","LINESTRING (80.22913 12.91349, 80.231269999999...",297.4,3323.8
390,S15-197-27,"Karapakkam,N S.K Street",Tenable,197,24,97,15,"Karapakkam,N S.K Street, Ward 197, SOZHANGANAL...",12.913490,80.229135,...,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,2601.931062,"{'route': [(12.91349, 80.22913), (12.92888, 80...","LINESTRING (80.22913 12.91349, 80.231269999999...",297.4,3323.8
391,S15-197-21,Karapakkam,Tenable,197,49,178,15,"Karapakkam, Ward 197, SOZHANGANALLUR, Chennai,...",12.919052,80.230034,...,195,Kannagi Nagar,"Urban Primary Health Centre, Kannagi Nagar, Ne...",12.926553,80.239799,1959.684015,"{'route': [(12.91905, 80.23008), (12.91715, 80...","LINESTRING (80.23008 12.91905, 80.22982 12.917...",264.9,3122.4


In [6]:
#create a geodataframe and pass the geometry column, created using extract_route() method, as the geometry information of the geodataframe
gdf = gpd.GeoDataFrame(df, geometry = df['geometry'])

#Export in whichever spatial formats you need to
gdf.to_file('OSRM_FastestRoutesToNearestFacility.shp')
gdf.to_file('OSRM_FastestRoutesToNearestFacility.geojson', driver= 'GeoJSON')

# 2. OSMnx and Networkx for shortest paths

OSMnx is a very handy tool for network analysis which also allows you to easily access and import road network graph objects based on OSM data. OSMnx library allows you quite a lot of control over the graph behavior and a neat interface to model the networks. Read more about this project on its [documentation](https://osmnx.readthedocs.io/en/stable/#) page for all its modules and also [here](https://geoffboeing.com/2016/11/osmnx-python-street-networks/). 

In this example, We use OSMnx 
1) to import street graph for the Greater Chennai Corporation (GCC) region (data source used for GCC boundary - [Datameet (2016)](https://github.com/datameet/Municipal_Spatial_Data/tree/master/Chennai) ) <br/>
2) to find nearest network node for each origin and destination point and 

We use Networkx to calculate the shortest path using the graph built by OSMnx.

In [ ]:
GCC_boundary = "GCC.shp"

file = ogr.Open(cma_boundary)
layer = file.GetLayer(0)
feature = layer.GetFeature(0)
geom= feature.GetGeometryRef()
feature_json = geom.ExportToJson()

import json
loadasdict = json.loads(feature_json)
print(loadasdict)
#to shapely object
geom2 = shape(loadasdict) # or shp_geom = shape(first) with PyShp
#print(geom)
G = ox.graph_from_polygon(geom2)
G2 = ox.consolidate_intersections(G, tolerance=10, rebuild_graph=True, clean_periphery = True, dead_ends=True)
ox.save_graph_geopackage(G, filepath='osmnx_gcc_road_network.gpkg')
print(G)

In [8]:
#Read the Origin-Destination csv
od_table = 'ODMatrix_Settlements_NearestFacility.csv'
df = pd.read_csv(od_table)

The methods below are built based on the steps outlined by [Boeing (2019)](https://stackoverflow.com/a/58311118/7105292).

In [9]:
def nodes_to_linestring(path):
    coords_list = [(G.nodes[i]['x'], G.nodes[i]['y']) for i in path ]
    #print(coords_list)
    line = LineString(coords_list)
    
    return(line)

def shortestpath(o_lat, o_long, d_lat, d_long):
    
    nearestnode_origin, dist_o_to_onode = ox.distance.get_nearest_node(G, (o_lat, o_long), method='haversine', return_dist=True)
    nearestnode_dest, dist_d_to_dnode = ox.distance.get_nearest_node(G, (d_lat, d_long), method='haversine', return_dist=True)
    
    #Add up distance to nodes from both o and d ends. This is the distance that's not covered by the network
    dist_to_network = dist_o_to_onode + dist_d_to_dnode
    
    shortest_p = nx.shortest_path(G,nearestnode_origin, nearestnode_dest) 
    
    route = nodes_to_linestring(shortest_p) #Method defined above
    
    # Calculating length of the route requires projection into UTM system. Using 
    inSpatialRef = osr.SpatialReference()
    inSpatialRef.ImportFromEPSG(4326)
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(32643)
    coordTransform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    #route.wkt returns wkt of the shapely object. This step was necessary as transformation can be applied 
    #only on an ogr object. Used EPSG 32643 as Bangalore is in 43N UTM grid zone.
    geom = ogr.CreateGeometryFromWkt(route.wkt)
   
    geom.Transform(coordTransform)
    length = geom.Length()
    
    #Total length to be covered is length along network between the nodes plus the distance from the O,D points to their nearest nodes
    total_length = length + dist_to_network
    #in metres
    
    return(route, total_length )

In [ ]:
start_time = time.time()

df['osmnx_geometry'] = df.apply(lambda x: shortestpath(x['s_lat'], x['s_lon'], x['u_lat'], x['u_lon'])[0] , axis=1)
df['osmnx_length'] = df.apply(lambda x: shortestpath(x['s_lat'], x['s_lon'], x['u_lat'], x['u_lon'])[1] , axis=1)

print("Time taken: ", (time.time() - start_time), "seconds")
df
#Note that the lambda function returns a tuple. While applying the function, have add [0] and [1] to return only one of the two outputs. 
#There must be a nicer way to add both outputs in one go. This was more of a fluke try which worked. Alternatively, you could define two functions to return both separtely, but might be an overkill

In [13]:
#rename osmnx_geometry column to 'geometry' to pass it as the geometry component to the new geo dataframe
df = df.rename(columns = {'osmnx_geometry': 'geometry'})
gpdf = gpd.GeoDataFrame(df, geometry =df['geometry'])
gpdf.to_file('osmnx_shortestpaths.shp')
